In [ ]:
# This function does what it's supposed to; however,
# it could easily be replaced with something like 
# annotations = clinical[['Histologic_Grade_FIGO', 'Diabetes]]
# Initially I was trying to get this to do the whole process
# interactively, so people would just input values and it would
# work, but I have since split off those parts to try and test them.
# They don't work inside this function (so far), and only some kind of
# worked outside this function. So I simplified. A LOT.

def returnSubsetDF(df):
    print('Here are your options')
    print(set(df))
    boundary_names = []
    print('Please input the columns you would like to binarize. Type "done" to finish.')
    boundary = ''
    while boundary != 'done':
        boundary = input()
        if boundary == 'done':
            break
        elif boundary not in set(df):
            print('Invalid input\nHere are your options:')
            print(set(df))
        else:
            boundary_names.append(boundary)
    new_df = pd.DataFrame()
    for boundary in boundary_names:
        new_df[boundary] = df[boundary]
    return(new_df)

smaller_df = returnSubsetDF(clinical)

In [ ]:
# While this is a great idea and all, I can't figure out how to do this correctly

def binarizeV2(df):
    print('The following columns have been chosen for binarization:\n')
    boundary_names = list(set(df))
    example_dict = {}
    print('Please provide a dictionary to binarize them in the following format:\n')
    for boundary in boundary_names:
          print(boundary)
          example_dict[boundary] = {'option1': ['values', 'that', 'correspond', 'with', 'option1'],
                                   'option2': ['values', 'that', 'correspond', 'with', 'option2'], 
                                    'NaN' : [np.nan]
                                   }
    print(example_dict)
    

def binarizeDictionary(df, boundary_dict):
    boundary_names = list(set(df))
    for boundary in boundary_names:
        for key, value in boundary_dict.items():
            new_col = df[boundary].where(cond=(df[boundary] not in value), other=key)
        new_df[boundary] = new_col
    return(new_df)

my_dictionary = {'Beginning_Stage' : ['IA', 'IB', 'II'], 
                 'Later_Stage' : ['IIIA', 'IIIB', 'IIIC1', 'IIIC2', 'IVB'], 
                 np.nan : [np.nan]
                }
binarizeDictionary(smaller_df, my_dictionary)                

In [ ]:
#Note: THESE WORK AS EXPECTED. However, they are unnecessarily complicated,
#and have been replaced with simpler, more intuitive approachs.

def binarizeCategorical(column, option1, option1_list, option2, option2_list):
    """Input parameters:
           
           column: 
               A column in the clinical dataframe
           
           option1:
               This is the value that will replace original
               values that are within your option1_list.
           
           option1_list: 
               This is the list of values that the original
               value will be compared against. If that value
               is within option1_list, it will be replaced
               by option1.
           
           option2:
               This is the value that will replace original
               values that are within your option2_list.
           
           option2_list: 
               This is the list of values that the original
               value will be compared against. If that value
               is within option2_list, it will be replaced
               by option2.
       
       Method description:
           This function is used for separating columns with 
           multiple categorical values, such as 'Tumor_Site_Other',
           into two options, into one of two user-defined groups.
           
           It should replace the original value in the 
           clinical dataframe with option1 if the original
           value is found in option1_list. Otherwise, it will
           replace the original value with option2 if the
           original value in option2_list. Otherwise, it will 
           not change the original value. It should only return 
           the unchanged, original value if that value is 'NaN'.
       
       Return Value:
           The return value should be a list of values with two
           options, either option1 or option2. This function is 
           for columns with categorical variables, such as 
           clinical['Histologic_grade_FIGO'] or clinical['Race'].
    """
    return [ option1 if (x in option1_list) else option2 if (x in option2_list) else x for x in column ]

def binarizeCutOff(column, cut_off, replace_low, replace_high):
    """Input parameters:
           
           column: 
               A column in the clinical dataframe
           
           cut_off:
               The bar at which you compare the original
               samples in the dataframe. For example, you 
               may set a cut_off of 65 for the clinical['Age'] 
               column to separate Retired and Working individuals. 
           
           replace_low: 
               The value that will replace original values
               lower than cut_off in the chosen column.
           
           replace_high: 
               The value that will replace original values
               greater than cut_off in the chosen column.
       
       Method description:
           This should replace the original value in the 
           clinical dataframe with replace_low if the original
           value is lower than cut_off. Otherwise, it will
           replace the original value with replace_high if the
           original value is greater than or equal to cut_off.
           Otherwise, it will not change the original value. 
           Theoretically, it should only return the unchanged,
           original value if that value is 'NaN'.
       
       Return Value:
           The return value should be a list of values with two
           options, either replace_high or replace_low. This
           function is for columns with continuous variables, 
           such as integers or floats
    """
    return [ replace_low if (x < cut_off) else replace_high if x >= cut_off else x for x in column ]

def binarizeRange(column, low_cut_off, high_cut_off, in_range='In_Range', out_of_range='Out_Of_Range'):
    """Input parameters:
           
           column: 
               A column in the clinical dataframe
           
           low_cut_off:
               The lowest value of a researchers's specified
               range. For example, a value of 18.5 for BMI
               could be the function's low_cut_off.
           
           high_cut_off: 
               The highest value of a researchers's specified
               range. For example, a value of 24.9 for BMI
               could be the function's high_cut_off.
                      
           in_range: 
               The value that will replace original values
               within the specified range between low_cut_off
               and high_cut_off.
               
           out_of_range:
               The value that will replace original values
               outside of the specified range between
               low_cut_off and high_cut_off.
       
       Method description:
           This should replace the original value in the 
           clinical dataframe with in_range if the original
           value is within the range from low_cut_off to 
           high_cut_off. Otherwise, it will replace the 
           original value with out_of_range if the original 
           value is not within that range. Otherwise, it 
           will not change the original value. It should only 
           return the unchanged, original value if it is 'NaN'.
       
       Return Value:
           The return value should be a list of values with
           two options, either in_range or out_of_range. This
           function is for columns with continuous variables, 
           such as integers or floats
    """
    return [ in_range if (x >= low_cut_off and x <= high_cut_off) else out_of_range  
            if (x < low_cut_off or x > high_cut_off) else x for x in column ]